In [1]:
%load_ext autoreload
%autoreload 2

In [6]:
from ceres_infer.session import workflow
from ceres_infer.models import model_infer

In [7]:
import logging
logging.basicConfig(level=logging.INFO)

In [8]:
params = {
    # directories
    'outdir_run': '../out/20.0518 Lx/L0only_reg_dummy/', # output dir for the run
    'outdir_modtmp': '../out/20.0518 Lx/L0only_reg_dummy/model_perf/', # intermediate files for each model
    'indir_dmdata_Q3': '../out/20.0817 proc_data/gene_effect/dm_data.pkl', # pickled preprocessed DepMap Q3 data
    'indir_dmdata_Q4': '../out/20.0817 proc_data/gene_effect/dm_data_Q4.pkl', # pickled preprocessed DepMap Q3 data
    'indir_genesets': '../data/gene_sets/',
    'indir_landmarks': '../out/19.1013 tight cluster/landmarks_n200_k200.csv', # csv file of landmarks [default: None]

    # notes
    'session_notes': 'dummy regressor (median); predicting whole-genome',

    # data
    'opt_scale_data': False, # scale input data True/False
    'opt_scale_data_types': '\[(?:RNA-seq|CN)\]', # data source types to scale; in regexp
    'model_data_source': ['CERES_Lx'],
    'anlyz_set_topN': 10, # for analysis set how many of the top features to look at
    'perm_null': 1000, # number of samples to get build the null distribution, for corr
    'useGene_dependency': False, # whether to use CERES gene dependency (true) or gene effect (false)
    'scope': 'all', # scope for which target genes to run on; list of gene names, or 'all', 'differential'

    # model
    'model_name': 'dummy_reg',
    'model_params': {},
    'model_paramsgrid': {},
    'model_pipeline': model_infer,
    
    # analysis
    'metric_eval': 'score_test',  # metric in model_results to evaluate, e.g. score_test, score_oob
    'thresholds': {'score_rd10': 0.1,  # score of reduced model - threshold for filtering
                   'recall_rd10': 0.95},  # recall of reduced model - threshold for filtering
    'min_gs_size': 4 # minimum gene set size, to be derived
}

In [ ]:
wf = workflow(params)
pipeline = ['load_processed_data', 'infer']
wf.create_pipe(pipeline)
wf.run_pipe()

INFO:root:Loading preprocessed data...
INFO:root:Adding landmarks...
INFO:root:Running model building and inference...
  0%|          | 0/18141 [00:00<?, ?it/s]/Users/boyangzhao/anaconda/envs/cnp/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/boyangzhao/anaconda/envs/cnp/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/boyangzhao/anaconda/envs/cnp/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/Users/boyangzhao/anaconda/envs/cnp/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/Users/boyangzhao/anaconda/envs/cnp/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1912: RuntimeWar

  0%|          | 9/18141 [00:13<7:20:12,  1.46s/it]/Users/boyangzhao/anaconda/envs/cnp/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/boyangzhao/anaconda/envs/cnp/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/boyangzhao/anaconda/envs/cnp/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/Users/boyangzhao/anaconda/envs/cnp/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/Users/boyangzhao/anaconda/envs/cnp/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)
  0%|          | 10/18141 [00:14<7:

  0%|          | 18/18141 [00:27<9:04:45,  1.80s/it]/Users/boyangzhao/anaconda/envs/cnp/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/boyangzhao/anaconda/envs/cnp/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/boyangzhao/anaconda/envs/cnp/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/Users/boyangzhao/anaconda/envs/cnp/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/Users/boyangzhao/anaconda/envs/cnp/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)
  0%|          | 19/18141 [00:29<1

  0%|          | 27/18141 [00:41<7:16:47,  1.45s/it]/Users/boyangzhao/anaconda/envs/cnp/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/boyangzhao/anaconda/envs/cnp/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/boyangzhao/anaconda/envs/cnp/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/Users/boyangzhao/anaconda/envs/cnp/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/Users/boyangzhao/anaconda/envs/cnp/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)
  0%|          | 28/18141 [00:43<7

  0%|          | 36/18141 [00:55<6:40:49,  1.33s/it]/Users/boyangzhao/anaconda/envs/cnp/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/boyangzhao/anaconda/envs/cnp/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/boyangzhao/anaconda/envs/cnp/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/Users/boyangzhao/anaconda/envs/cnp/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/Users/boyangzhao/anaconda/envs/cnp/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)
  0%|          | 37/18141 [00:56<6

...

  5%|▌         | 927/18141 [22:24<7:15:18,  1.52s/it]/Users/boyangzhao/anaconda/envs/cnp/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/boyangzhao/anaconda/envs/cnp/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/boyangzhao/anaconda/envs/cnp/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/Users/boyangzhao/anaconda/envs/cnp/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/Users/boyangzhao/anaconda/envs/cnp/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)
  5%|▌         | 928/18141 [22:25

  5%|▌         | 936/18141 [22:35<5:56:38,  1.24s/it]/Users/boyangzhao/anaconda/envs/cnp/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/boyangzhao/anaconda/envs/cnp/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/boyangzhao/anaconda/envs/cnp/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/Users/boyangzhao/anaconda/envs/cnp/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/Users/boyangzhao/anaconda/envs/cnp/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)
  5%|▌         | 937/18141 [22:36

  5%|▌         | 945/18141 [22:46<5:44:41,  1.20s/it]/Users/boyangzhao/anaconda/envs/cnp/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/boyangzhao/anaconda/envs/cnp/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/boyangzhao/anaconda/envs/cnp/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/Users/boyangzhao/anaconda/envs/cnp/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/Users/boyangzhao/anaconda/envs/cnp/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)
  5%|▌         | 946/18141 [22:47

  5%|▌         | 954/18141 [22:57<6:03:28,  1.27s/it]/Users/boyangzhao/anaconda/envs/cnp/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/boyangzhao/anaconda/envs/cnp/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/boyangzhao/anaconda/envs/cnp/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/Users/boyangzhao/anaconda/envs/cnp/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/Users/boyangzhao/anaconda/envs/cnp/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)
  5%|▌         | 955/18141 [22:58

  5%|▌         | 963/18141 [23:08<5:51:12,  1.23s/it]/Users/boyangzhao/anaconda/envs/cnp/lib/python3.7/site-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/Users/boyangzhao/anaconda/envs/cnp/lib/python3.7/site-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]
/Users/boyangzhao/anaconda/envs/cnp/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/Users/boyangzhao/anaconda/envs/cnp/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/Users/boyangzhao/anaconda/envs/cnp/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)
  5%|▌         | 964/18141 [23:09

In [ ]:
wf = workflow(params)
pipeline = ['load_processed_data', 'load_model_results', 'analyze', 'analyze_filtered', 'derive_genesets']
wf.create_pipe(pipeline)
wf.run_pipe()